In [ ]:
import os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from delta import *
import findspark

findspark.init()

warehouse_diretory_path = '[YOUR_WAREHOUSE]'
drugbank_drug_annotation_file = 'file:///[FILE_FULL_NAME:drugbank_all_full_database.xml]'
biological_database_name = 'biological_database'
drugbank_drug_annotation_table_name = 'bronze_drugbank_drug_annotation'

conf = SparkConf()
conf.setAll(
    [
        ('spark.master', 'local[*]'), 
        ('spark.driver.host', 'localhost'),
        ('spark.app.name', 'DrugBank Drug Annotation Importer'),
        ('spark.ui.showConsoleProgress', 'true'),
        ('spark.sql.execution.arrow.pyspark.enabled', 'false'),
        ('spark.storage.memoryFraction', '0.6'),
        ('spark.driver.maxResultSize', '6g'),
        ('spark.driver.memory', '6g'),
        ('spark.executor.memory', '6g'),           
        ('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true'),
        ('spark.dynamicAllocation.enabled', 'false'),
        ('spark.sql.caseSensitive', 'true'),
        ('spark.sql.adaptive.enabled', 'true'),
        ('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension'),
        ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
        ('spark.sql.warehouse.dir', warehouse_diretory_path),
        ('spark.driver.extraJavaOptions', f'-Dderby.system.home={warehouse_diretory_path}')
    ])

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
drugbank_drug_annotation_df = spark.read \
    .format('xml') \
    .options(rowTag='drug') \
    .load(drugbank_drug_annotation_file)

In [ ]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {biological_database_name};')

In [ ]:
spark.sql(f'USE {biological_database_name};')

In [ ]:
drugbank_drug_annotation_df.write \
    .format('delta') \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .option('partitionOverwriteMode', 'dynamic') \
    .saveAsTable(drugbank_drug_annotation_table_name)

In [ ]:
spark.stop()